In [ ]:
import geopandas as gpd
from lonboard import Map, ScatterplotLayer
from lonboard.colormap import apply_continuous_cmap
import numpy as np
import pandas as pd
import matplotlib
from palettable.matplotlib import Viridis_10
from io import BytesIO
import plotly.express as px
import requests
import json
import os

In [ ]:
value = os.getenv('D4SCIENCE_TOKEN')
TOKEN = value

In [ ]:
BEACON_INSTANCE_URL = "https://beacon-wb1-ts.d4science.org/"

In [4]:
json_query = json.dumps(
    {
        "query_parameters": [
            {
                "column": "COMMON_TIME",
                "alias": "time"
            },
            {
                "column": "COMMON_TEMPERATURE",
                "alias": "temperature",
            },
            {
                "column": "COMMON_TEMPERATURE_P01",
                "alias": "temperature_p01",
            },
            {
                "column": "COMMON_TEMPERATURE_P06",
                "alias": "temperature_p06",
            },
            {
                "column": "COMMON_SALINITY",
                "alias": "salinity",
            },
            {
                "column": "COMMON_SALINITY_P01",
                "alias": "salinity_p01",
            },
            {
                "column": "COMMON_SALINITY_P06",
                "alias": "salinity_p06",
            },
            {
                "column": "COMMON_DEPTH",
                "alias": "depth",
            },
            {
                "column": "COMMON_LONGITUDE",
                "alias": "longitude",
            },
            {
                "column": "COMMON_LATITUDE",
                "alias": "latitude",
            },
            {
                "column": "SOURCE_BDI",
                "alias": "source_bdi",
            },
            {
                "column": "COMMON_PLATFORM_L06",
                "alias": "platform_l06",
            },
        ],
        "filters": [
            {
                "for_query_parameter": "depth", 
                "max": 10,
                "min": 0
            },
            {
                "for_query_parameter": "temperature", 
                "max": 40,
                "min": -5
            },
            {
                "for_query_parameter": "salinity", 
                "max": 100,
                "min": -5
            },
            {
                'is_not_null': {
                    'for_query_parameter': 'depth',
                }
            },
            {
                "for_query_parameter": "time",
                "min": "2020-01-01T00:00:00Z",
                "max": "2020-12-31T00:00:00Z",
            },
        ],
        "output": {
            "format": "parquet"
        },
    }
)
headers = {
    "Content-Type": "application/json",
    "Authorization": "Bearer " + TOKEN,
}

response = requests.request("POST", f"{BEACON_INSTANCE_URL}/api/query", headers=headers, data=json_query)

In [5]:
response_bytes = BytesIO(response.content)
df = pd.read_parquet(response_bytes)
df['time'] = pd.to_datetime(df['time'], unit='s')

# Turn into a GeoDataFrame
gdf = gpd.GeoDataFrame(
    df,
    geometry=gpd.points_from_xy(df.longitude, df.latitude),
    crs="EPSG:4326"
)

In [6]:
gdf

,time,temperature,temperature_p01,temperature_p06,salinity,salinity_p01,salinity_p06,depth,longitude,latitude,source_bdi,platform_l06,geometry
0,2020-03-11 00:00:00,6.658000,SDN:P01::TEMPPR01,SDN:P06::UPAA,20.269001,SDN:P01::PSLTZZ01,SDN:P06::UUUU,1.000000,8.70480,53.8771,BEACON_CORA_TS,None,POINT (8.7048 53.8771)
1,2020-03-11 00:10:00,6.669000,SDN:P01::TEMPPR01,SDN:P06::UPAA,20.494001,SDN:P01::PSLTZZ01,SDN:P06::UUUU,1.000000,8.70480,53.8771,BEACON_CORA_TS,None,POINT (8.7048 53.8771)
2,2020-03-11 00:20:00,6.695000,SDN:P01::TEMPPR01,SDN:P06::UPAA,20.840000,SDN:P01::PSLTZZ01,SDN:P06::UUUU,1.000000,8.70480,53.8771,BEACON_CORA_TS,None,POINT (8.7048 53.8771)
3,2020-03-11 00:30:00,6.744000,SDN:P01::TEMPPR01,SDN:P06::UPAA,21.051001,SDN:P01::PSLTZZ01,SDN:P06::UUUU,1.000000,8.70480,53.8771,BEACON_CORA_TS,None,POINT (8.7048 53.8771)
4,2020-03-11 00:40:00,6.780000,SDN:P01::TEMPPR01,SDN:P06::UPAA,21.332001,SDN:P01::PSLTZZ01,SDN:P06::UUUU,1.000000,8.70480,53.8771,BEACON_CORA_TS,None,POINT (8.7048 53.8771)
...,...,...,...,...,...,...,...,...,...,...,...,...,...
23155663,2020-01-21 01:02:03,29.497999,SDN:P01::TEMPPR01,SDN:P06::UPAA,34.645100,SDN:P01::PSLTZZ01,SDN:P06::UUUU,9.146668,119.18202,-13.4197,BEACON_CORA_PR,None,POINT (119.18202 -13.4197)
23155664,2020-01-21 13:24:02,19.423000,SDN:P01::TEMPPR01,SDN:P06::UPAA,35.258999,SDN:P01::PSLTZZ01,SDN:P06::UUUU,4.000908,84.72940,-34.8902,BEACON_CORA_PR,None,POINT (84.7294 -34.8902)
23155665,2020-01-21 13:24:02,19.421000,SDN:P01::TEMPPR01,SDN:P06::UPAA,35.258301,SDN:P01::PSLTZZ01,SDN:P06::UUUU,5.986441,84.72940,-34.8902,BEACON_CORA_PR,None,POINT (84.7294 -34.8902)
23155666,2020-01-21 13:24:02,19.413000,SDN:P01::TEMPPR01,SDN:P06::UPAA,35.259300,SDN:P01::PSLTZZ01,SDN:P06::UUUU,7.971954,84.72940,-34.8902,BEACON_CORA_PR,None,POINT (84.7294 -34.8902)


In [7]:
import plotly.graph_objects as go
# Ensure time is datetime
gdf["time"] = pd.to_datetime(gdf["time"])

# Resample by day for each source_bdi and calculate mean temperature
mean_temp = (
    gdf.groupby([pd.Grouper(key="time", freq="D"), "source_bdi"])["temperature"]
    .mean()
    .reset_index()
)

# Plot with a line per source_bdi
fig = px.line(
    mean_temp,
    x="time",
    y="temperature",
    color="source_bdi",  # different line per source_bdi
    title="Daily Mean Sea Water Temperature [0-5m] by Source",
    labels={"time": "Day", "temperature": "Mean Temperature (°C)", "source_bdi": "Source"},
    height=500
)

fig.update_layout(yaxis_range=[0, 40])
fig.show()

# === Salinity: Mean by day and source_bdi ===
mean_sal = (
    gdf.groupby([pd.Grouper(key="time", freq="D"), "source_bdi"])["salinity"]
    .mean()
    .reset_index()
)

fig_sal = px.line(
    mean_sal,
    x="time",
    y="salinity",
    color="source_bdi",
    title="Daily Mean Sea Water Salinity [0-5m] by Source",
    labels={"time": "Day", "salinity": "Mean Salinity (PSU)", "source_bdi": "Source"},
    height=400
)
fig_sal.update_layout(yaxis_range=[0, 40])
fig_sal.show()